In [ ]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 198 kB 69.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=f78ae808ecd44b95ac146d024dc6b5cb66e95ea7208a025af74ea42cc978133d
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [ ]:
spark

In [ ]:
df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/test1.csv', header=True)

In [ ]:
df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2|18000 |
+---------+---+----------+------+



In [ ]:
df.count()

6

In [ ]:
df.select('name','age').show()

+---------+---+
|     name|age|
+---------+---+
|    Krish| 31|
|Sudhanshu| 30|
|    Sunny| 29|
|     Paul| 24|
|   Harsha| 21|
|  Shubham| 23|
+---------+---+



In [ ]:
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)



In [ ]:
df.withColumn('Age',df['age'].cast('Integer')).printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)



In [27]:
df.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2|18000 |
+---------+---+----------+------+



In [47]:
from datetime import datetime,timedelta
import pytz
from pyspark.sql.functions import col,lit
from pyspark.sql.functions import *

In [32]:
tz_mexico = pytz.timezone('America/Mexico_City')
fecha_hoy = datetime.now(tz_mexico).strftime('%Y-%m-%d %H:%M:%S')

In [33]:
print(fecha_hoy)

2022-05-21 21:26:15


In [41]:
dff = df.withColumn('Fecha', lit(fecha_hoy))
dff.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|              Fecha|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|2022-05-21 21:26:15|
|Sudhanshu| 30|         8| 25000|2022-05-21 21:26:15|
|    Sunny| 29|         4| 20000|2022-05-21 21:26:15|
|     Paul| 24|         3| 20000|2022-05-21 21:26:15|
|   Harsha| 21|         1| 15000|2022-05-21 21:26:15|
|  Shubham| 23|         2|18000 |2022-05-21 21:26:15|
+---------+---+----------+------+-------------------+



In [42]:
dff.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- Fecha: string (nullable = false)



In [44]:
fecha_hoy_dt = datetime.now(tz_mexico)

In [76]:
df_unix = dff.select('Name','age','Experience','Salary',unix_timestamp(col('Fecha')).alias('fecha'))
df_unix.show()

+---------+---+----------+------+----------+
|     Name|age|Experience|Salary|     fecha|
+---------+---+----------+------+----------+
|    Krish| 31|        10| 30000|1653168375|
|Sudhanshu| 30|         8| 25000|1653168375|
|    Sunny| 29|         4| 20000|1653168375|
|     Paul| 24|         3| 20000|1653168375|
|   Harsha| 21|         1| 15000|1653168375|
|  Shubham| 23|         2|18000 |1653168375|
+---------+---+----------+------+----------+



In [78]:
df_not_unix = df_unix.select('Name','age','Experience','Salary',from_unixtime(col('Fecha')).alias('fecha'))
df_not_unix.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|              fecha|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|2022-05-21 21:26:15|
|Sudhanshu| 30|         8| 25000|2022-05-21 21:26:15|
|    Sunny| 29|         4| 20000|2022-05-21 21:26:15|
|     Paul| 24|         3| 20000|2022-05-21 21:26:15|
|   Harsha| 21|         1| 15000|2022-05-21 21:26:15|
|  Shubham| 23|         2|18000 |2022-05-21 21:26:15|
+---------+---+----------+------+-------------------+



In [79]:
df_convert = df_not_unix.withColumn("fecha", unix_timestamp(col('fecha')).cast('bigint'))
df_convert.show()
df_convert.printSchema()

+---------+---+----------+------+----------+
|     Name|age|Experience|Salary|     fecha|
+---------+---+----------+------+----------+
|    Krish| 31|        10| 30000|1653168375|
|Sudhanshu| 30|         8| 25000|1653168375|
|    Sunny| 29|         4| 20000|1653168375|
|     Paul| 24|         3| 20000|1653168375|
|   Harsha| 21|         1| 15000|1653168375|
|  Shubham| 23|         2|18000 |1653168375|
+---------+---+----------+------+----------+

root
 |-- Name: string (nullable = true)
 |-- age: string (nullable = true)
 |-- Experience: string (nullable = true)
 |-- Salary: string (nullable = true)
 |-- fecha: long (nullable = true)

